In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
filename = 'C:/Users/Anna Marie/Downloads/filt_mig.sgy'

In [6]:
from obspy.io.segy.segy import _read_segy
stream = _read_segy(filename, headonly=True)

In [7]:

traces_with_indexes = []

for trace in stream.traces[:10000]:
    x = trace.header.x_coordinate_of_ensemble_position_of_this_trace - 1
    y = trace.header.y_coordinate_of_ensemble_position_of_this_trace - 1

    traces_with_indexes.append((x,y, trace.data))


In [8]:
x_max = max(traces_with_indexes, key = lambda x: x[0])[0]
y_max = max(traces_with_indexes, key = lambda x: x[1])[1]

print(x_max, y_max)

53 187


In [9]:
x_len = x_max + 1
y_len = y_max + 1
s_len = len(traces_with_indexes[0][2])
threeDSamples = np.zeros((x_len, y_len, s_len))

In [10]:
for xy_trace in traces_with_indexes:
    x = xy_trace[0]
    y = xy_trace[1]
    samples = xy_trace[2]
    threeDSamples[x, y] = samples

In [70]:
vm = np.percentile(threeDSamples, 99)

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
samples   = stream.binary_file_header.number_of_samples_per_data_trace
samples = samples -1

def f(x):
    plt.figure(figsize=(16,8))
    plt.imshow(threeDSamples[x].T, cmap="seismic", vmin=-vm, vmax=vm, aspect=0.25)
    plt.colorbar(shrink=0.5)
    plt.show()
    #return x
#interact(myplot, start = 0, x_max=10);
interact(f, x=widgets.IntSlider(min=0,max=x_max,step=1,value=10));


def ff(x):
    plt.figure(figsize=(16,8))
    plt.imshow(threeDSamples[:,x,:].T, cmap="seismic", vmin=-vm, vmax=vm, aspect=0.25)
    plt.colorbar(shrink=0.5)
    plt.show()
    #return x
#interact(myplot, start = 0, x_max=10);
interact(ff, x=widgets.IntSlider(min=0,max=y_max,step=1,value=10));

def fff(x):
    plt.figure(figsize=(16,8))
    plt.imshow(threeDSamples[:,:,x].T, cmap="gray", vmin=-vm, vmax=vm, aspect=0.25)
    plt.colorbar(shrink=0.5)
    plt.show()
    #return x
#interact(myplot, start = 0, x_max=10);
interact(fff, x=widgets.IntSlider(min=0,max=samples,step=1,value=10));


interactive(children=(IntSlider(value=10, description='x', max=53), Output()), _dom_classes=('widget-interact'…

interactive(children=(IntSlider(value=10, description='x', max=187), Output()), _dom_classes=('widget-interact…

interactive(children=(IntSlider(value=10, description='x', max=1500), Output()), _dom_classes=('widget-interac…